In [1]:
from data_utils import get_CIFAR10_data
import numpy as np

import keras
import tensorflow as tf
from keras.layers import Input, Conv2D, Dense, Flatten, Add, ZeroPadding2D, Merge, add, MaxPool2D
from keras.models import Model
from keras.utils.np_utils import to_categorical 
import tensorflow_weave
reload(tensorflow_weave)
from tensorflow_weave import *

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [27]:
filter_size = 3
num_filters = 2
pad_size = filter_size//2
mid_layer = 100


X_train = data['X_train']
Y_train = data['y_train']
X_Val = data['X_val']
Y_Val = data['y_val']

Y_train = to_categorical(Y_train, num_classes=10)
Y_val = to_categorical(Y_Val, num_classes=10)

In [45]:
def build_weave_conv_model(num_filters1, num_filters2):
    inputs = Input(shape=(3,32,32))


    ######### Layer 1
    x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)


    x_per = Conv2D(num_filters1,
                   kernel_size = (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation='relu')(x_pad)

    x_loc = Conv2D(num_filters1,
                   kernel_size= (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation = 'relu')(x_pad)

    x_zero = ZeroWeave()(x_loc)
    x_weave = ArrayWeave()(x_per)

    x = Add()([x_weave, x_zero])

    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

    x = Conv2D(num_filters2,
               kernel_size= (filter_size,filter_size),
               strides=(filter_size,filter_size),
               padding='valid',
               activation = 'relu')(x)
    x = MaxPool2D()(x)
#     ###############
#     x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(x)


#     x_per = Conv2D(2*num_filters,
#                    kernel_size = (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation='relu')(x_pad)

#     x_loc = Conv2D(2*num_filters,
#                    kernel_size= (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation = 'relu')(x_pad)

#     x_zero = ZeroWeave()(x_loc)
#     x_weave = ArrayWeave()(x_per)

#     x = Add()([x_weave, x_zero])

#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                kernel_size= (filter_size,filter_size),
#                strides=(filter_size,filter_size),
#                padding='valid',
#                activation = 'relu')(x)

#     x = MaxPool2D()(x)
    x = Flatten()(x)
    x = Dense(mid_layer, activation = 'relu')(x)
    predictions = Dense(10, activation='softmax')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=[inputs], outputs=predictions)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [42]:
#hist1 = model.fit(X_train, Y_train, epochs = 10, validation_split=.05, verbose=1)

Train on 46550 samples, validate on 2450 samples
Epoch 1/10
46550/46550 [==============================] - 432s 9ms/step - loss: 1.8561 - acc: 0.3450 - val_loss: 1.5603 - val_acc: 0.4359
Epoch 2/10
46550/46550 [==============================] - 413s 9ms/step - loss: 1.4973 - acc: 0.4614 - val_loss: 1.4424 - val_acc: 0.4845
Epoch 3/10
46550/46550 [==============================] - 386s 8ms/step - loss: 1.3901 - acc: 0.5018 - val_loss: 1.4278 - val_acc: 0.4853
Epoch 4/10
46550/46550 [==============================] - 384s 8ms/step - loss: 1.3142 - acc: 0.5273 - val_loss: 1.3253 - val_acc: 0.5310
Epoch 5/10
46550/46550 [==============================] - 380s 8ms/step - loss: 1.2548 - acc: 0.5502 - val_loss: 1.3510 - val_acc: 0.5216
Epoch 6/10
46550/46550 [==============================] - 383s 8ms/step - loss: 1.2092 - acc: 0.5682 - val_loss: 1.3272 - val_acc: 0.5229
Epoch 7/10
46550/46550 [==============================] - 386s 8ms/step - loss: 1.1706 - acc: 0.5803 - val_loss: 1.2908 - v

In [46]:
def build_norm_conv_model(num_filters1, num_filters2):
    inputs = Input(shape=(3,32,32))

    ####### Layer 1
    x = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)

    x = Conv2D(num_filters1,
                   kernel_size = (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation='relu')(x)

    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

    x = Conv2D(num_filters1,
                   kernel_size= (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation = 'relu')(x)

    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

    x = Conv2D(num_filters2,
               kernel_size= (filter_size,filter_size),
               strides=(1,1),
               padding='valid',
               activation = 'relu')(x)

    x = MaxPool2D()(x)
#     ####### Layer 2
#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                    kernel_size = (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation='relu')(x)

#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                    kernel_size= (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation = 'relu')(x)

#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                kernel_size= (filter_size,filter_size),
#                strides=(1,1),
#                padding='valid',
#                activation = 'relu')(x)

#     x = MaxPool2D()(x)
    x = Flatten()(x)
    x = Dense(mid_layer, activation = 'relu')(x)
    predictions = Dense(10, activation='softmax')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model2 = Model(inputs=[inputs], outputs=predictions)
    model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model2

In [44]:
#hist2 = model2.fit(X_train, Y_train, epochs = 10, validation_split=.05, verbose=1)

Train on 46550 samples, validate on 2450 samples
Epoch 1/10
46550/46550 [==============================] - 201s 4ms/step - loss: 1.7015 - acc: 0.3862 - val_loss: 1.5456 - val_acc: 0.4555
Epoch 2/10
46550/46550 [==============================] - 192s 4ms/step - loss: 1.4392 - acc: 0.4834 - val_loss: 1.3541 - val_acc: 0.5192
Epoch 3/10
46550/46550 [==============================] - 194s 4ms/step - loss: 1.3289 - acc: 0.5256 - val_loss: 1.2541 - val_acc: 0.5604
Epoch 4/10
46550/46550 [==============================] - 194s 4ms/step - loss: 1.2407 - acc: 0.5608 - val_loss: 1.2223 - val_acc: 0.5710
Epoch 5/10
46550/46550 [==============================] - 186s 4ms/step - loss: 1.1683 - acc: 0.5828 - val_loss: 1.2024 - val_acc: 0.5767
Epoch 6/10
46550/46550 [==============================] - 192s 4ms/step - loss: 1.1108 - acc: 0.6024 - val_loss: 1.1369 - val_acc: 0.5902
Epoch 7/10
46550/46550 [==============================] - 198s 4ms/step - loss: 1.0707 - acc: 0.6185 - val_loss: 1.1749 - v

In [54]:
filters1 = [1,2]#,4,8,12]
filters2 = [1,2]#,4,8,12]
num_epochs = 1
weave_acc = {}
norm_acc = {}
for filt1 in filters1:
    for filt2 in filters2:
        print(filt1,filt2)
        weave_model = build_weave_conv_model(filt1,filt2)
        norm_model = build_norm_conv_model(filt1,filt2)
        weave_model.fit(X_train[0:100], Y_train[0:100], epochs = num_epochs, verbose=0)
        norm_model.fit(X_train[0:100], Y_train[0:100], epochs = num_epochs, verbose=0)
        weave_acc[(filt1,filt2)] = weave_model.evaluate(X_Val, Y_val, verbose = 0)[1]
        norm_acc[(filt1,filt2)] = norm_model.evaluate(X_Val, Y_val, verbose = 0)[1]
        print('Weave Model', weave_acc[(filt1,filt2)])
        print('Norm Model', norm_acc[(filt1,filt2)])

(1, 1)
('Weave Model', 0.096)
('Norm Model', 0.108)
(1, 2)
('Weave Model', 0.1)
('Norm Model', 0.084)
(2, 1)


KeyboardInterrupt: 